In [ ]:
import os
import requests
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(
    temperature=0.1,
)

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")

class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query : str = Field(
        description="The Query you will search for. Example quesry: Stock Market Symbol for Apple Company"
    )

class StockMarketSymbolSearchTool(BaseTool):
    name : str ="StockMarketSymbolSearchTool"
    description : str = """
    Use this toll to find the stock market symbol for a company.
    It takes a query as an argument.
    """
    args_schema : Type[StockMarketSymbolSearchToolArgsSchema] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


class CompanyOverviewArgsSchema(BaseModel):
    symbol : str = Field(description="Stock symbol of the company. Examples: AAPL, TSLA",)

class CompanFinancialOverviewTool(BaseTool):
    name: str="CompanyOverview"
    description : str = """
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema : Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=Overview&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()

class CompanyIncomeStatementTool(BaseTool):
    name : str = "CompanyIncomeStatement"
    description : str = """
    Use this to get the income statement of a company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get (f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()["annualReports"]

class CompanyStockPerformanceTool(BaseTool):
    name : str = "CompanyStockPerformance"
    description : str = """
    Use this to get the weekly Company stock persformance.
    You should enter a stock symbol
    """
    args_schema : Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=Time_SERIES_MONTHLY_Adjusted&symbol={symbol}&apikey={alpha_vantage_api_key}")

        return r.json()["Monthly Adjusted Time Series"]



agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
        CompanFinancialOverviewTool(),
        CompanyIncomeStatementTool(),
        CompanyStockPerformanceTool(),
    ]
)

prompt = "Give me information on Beyond meat's stock, and help me analyze if it's a potential good investment considering its financials, income statement, stock performance."
agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Stock Market Symbol for Beyond Meat Company'}`


Get a real-time Beyond Meat, Inc. (BYND) stock price quote with breaking news, financials, statistics, charts and more. ... Beyond Meat, Inc., a plant-based meat company, engages in the development, manufacture, marketing, and sale of plant-based meat products under the Beyond brand name in the United States and internationally. ... Beyond Meat ... Beyond Meat Unveils Expanded Line of Beyond Steak®, One of the Fastest Growing and Top Selling Plant-Based Meat Products on the Market (1) GlobeNewswire Feb 6, 2025 2:00pm About Beyond Meat. Beyond Meat, Inc., a plant-based meat company, engages in the development, manufacture, marketing, and sale of plant-based meat products under the Beyond brand name in the United States and internationally. The company sells a range of plant-based meat products that replicates beef, pork, and poultry meats. In

{'input': "Give me information on Beyond meat's stock, and help me analyze if it's a potential good investment considering its financials, income statement, stock performance.",
 'output': "### Beyond Meat Inc. (BYND) Overview:\n- **Description:** Beyond Meat, Inc. is a food company that manufactures, markets, and sells plant-based meat products in the United States and internationally.\n- **Sector:** Manufacturing\n- **Industry:** Food and Kindred Products\n- **Market Capitalization:** $242,027,000\n- **Revenue (TTM):** $326,452,000\n- **Gross Profit (TTM):** $41,699,000\n- **Net Income Margin (TTM):** -0.491\n- **EPS (TTM):** -4.19\n- **PE Ratio:** Not available\n- **52-Week High:** $10.31\n- **52-Week Low:** $3.1\n- **Analyst Ratings:** 0 Strong Buy, 0 Buy, 6 Hold, 3 Sell, 2 Strong Sell\n\n### Financial Analysis:\n- **Income Statement (Latest):**\n  - **Total Revenue:** $323,921,000\n  - **Gross Profit:** -$82,655,000\n  - **Operating Income:** -$341,898,000\n  - **Net Income:** -$3